# Implementing "FX sentiment analysis with large language models" (Ballinari et al.)
This paper can be found at 

## Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, EarlyStoppingCallback
from peft import LoraConfig, prepare_model_for_kbit_training
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os
from huggingface_hub import login
from trl import SFTTrainer
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, classification_report
from tqdm import tqdm

## 1. Dataset Preprocessing

### 1.1. Filtering
- Load the datasets
- Drop articles with <20 words
- Remove duplicate articles 
- Convert time to datetime

In [ ]:
# Loading the dailyfx news articles dataset (Title,Author,Date,Full Text,URL)
df_news = pd.read_csv('datasets/news_articles/dailyfx_articles_012011-062024.csv')
df_news = df_news[['Title', 'Date', 'Full Text', 'URL']]

# Load the fxstreet news articles dataset into same df (Title,Date,Full Text,URL)
temp = pd.read_csv('datasets/news_articles/fxstreet_articles.csv')
temp = temp[['Title', 'Date', 'Full Text', 'URL']]
df_news = pd.concat([df_news, temp], ignore_index=True)

# # Load investing.com news articles dataset into same df (Title,Full Text,URL,Date,Author)
temp = pd.read_csv('datasets/news_articles/investingcom_finaldata_2011-062024.csv')
temp = temp[['Title', 'Date', 'Full Text', 'URL']]
df_news = pd.concat([df_news, temp], ignore_index=True)

In [49]:
# Drop articles with <20 words
df_news = df_news[df_news['Full Text'].str.split().str.len() >= 20]

# Remove duplicate articles
df_news = df_news.drop_duplicates(subset=['Full Text'])
df_news = df_news.drop_duplicates(subset=['Title'])
df_news = df_news.drop_duplicates(subset=['URL'])

# Convert time to datetime
df_news['Date'] = pd.to_datetime(df_news['Date'], utc=True)

# Only keep articles before 2020
# df_news = df_news[df_news['Date'] < pd.to_datetime('2020-01-01', utc=True)]

# Truncate articles to max 32,767 characters
df_news['Full Text'] = df_news['Full Text'].str[:32767]

# Randomly sample 30000 articles
df_news = df_news.sample(n=30000, random_state=42)

In [50]:
df_news['Date'] = (
    df_news['Date']                        
    .dt.tz_convert('America/New_York')     # Convert to NY time like paper
    .dt.tz_localize(None)                  # Remove timezone info to allow for merging later
    .dt.normalize()                        # Normalise to midnight
)

# Sort by date
df_news = df_news.sort_values(by='Date')

### 1.2. Creating mentioned_currency column
- Use regex to capture all the currencies used in an article
- Make use of common synomyms
- Filter articles that don't mention any of the G10 currencies 

In [51]:
# Dictionary mapping ISO codes to the regex patterns (synonyms) from Figure A.1
currency_synonyms = {
    "EUR": [r"EUR", r"Euro"],
    "USD": [r"USD", r"Dollar", r"Dollars", r"US Dollar", r"US-Dollar", r"U\.S\. Dollar", 
            r"US Dollars", r"US-Dollars", r"U\.S\. Dollars", r"Greenback"],
    "JPY": [r"JPY", r"Yen", r"Japanese Yen"],
    "GBP": [r"GBP", r"Pound", r"Pounds", r"Sterling", r"British Pound", r"British Pounds"],
    "AUD": [r"AUD", r"Australian Dollar", r"Australian Dollars", r"Aussie"],
    "CAD": [r"CAD", r"Canadian Dollar", r"Canadian Dollars"],
    "CHF": [r"CHF", r"Swiss Franc", r"Swiss Francs", r"Swissie"],
    "NZD": [r"NZD", r"New Zealand Dollar", r"New Zealand Dollars", r"Kiwi"],
    "NOK": [r"NOK", r"Norwegian Krone", r"Norwegian Kroner"],
    "SEK": [r"SEK", r"Swedish Krona", r"Swedish Kronor"]
}

# Get list of mentioned currencies from text
def get_mentioned_currencies(text):
    mentioned_currencies = list()

    for currency, patterns in currency_synonyms.items():
        for pattern in patterns:
            if re.search(pattern, text, re.IGNORECASE):
                mentioned_currencies.append(currency)
                break

    return mentioned_currencies

df_news['mentioned_currencies'] = df_news['Full Text'].apply(get_mentioned_currencies)

# Filter articles to keep only those where 'mentioned_currencies' is non empty
df_news = df_news[df_news['mentioned_currencies'].apply(lambda x: len(x) > 0)]

# So that it is incrimenting by 1 properly due to dropped values from before
df_news = df_news.reset_index(drop=True)

### 1.3. Getting historical prices
Using nominal narrow effective exchange rate (daily) for each country.

Narrow effective exchange rate is a good proxy for the tradable currency index that the authors used.

Allows us to put a number to the currency rather than using a pair as then the currency can be effected by the other in the pair. 

It is done by taking the geometric mean from the exchange rate of various other currencies (narrow means only a small number of industrialised countries so that the average isn't skewed by some other non industrialised country going down).

In [52]:
# All links to get data from for effective exchage rate
urls = {
    "USD": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.US?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "EUR": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.XM?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "JPY": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.JP?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "GBP": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.GB?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "CAD": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.CA?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "AUD": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.AU?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "CHF": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.CH?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv", 
    "SEK": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.SE?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "NOK": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.NO?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv",
    "NZD": "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_EER/1.0/D.N.N.NZ?startPeriod=2011-01-01&endPeriod=2024-06-01&format=csv"
}

# Initialise an empty DataFrame (EER = effective exchange rate)
df_EER = pd.DataFrame()

for code, url in urls.items():
    # Read only the required columns from the CSV
    df_temp = pd.read_csv(url, usecols=lambda c: c in ["TIME_PERIOD", "OBS_VALUE"])
    
    # Convert OBS_VALUE to float for log calculations later
    df_temp["OBS_VALUE"] = pd.to_numeric(df_temp["OBS_VALUE"], errors="coerce")
    
    # Rename "OBS_VALUE" to currency code
    df_temp = df_temp.rename(columns={
        "OBS_VALUE": code,
        "TIME_PERIOD": "date"
    })
    
    # If the main df is empty, set it to this df
    if df_EER.empty:
        df_EER = df_temp
    else:
        # Join on "date", keep all records (outer join)
        df_EER = pd.merge(df_EER, df_temp, on=["date"], how='outer')


df_EER['date'] = pd.to_datetime(df_EER['date'])
df_EER = df_EER.set_index('date')

# drop all NaNs in the data
df_EER.dropna(inplace=True)

df_EER.head()

,USD,EUR,JPY,GBP,CAD,AUD,CHF,SEK,NOK,NZD
date,,,,,,,,,,
2011-01-03,82.07,103.41,120.99,105.23,129.89,130.38,87.30,116.49,136.00,98.33
2011-01-04,82.05,103.98,119.81,105.84,129.59,128.97,86.05,116.47,135.74,97.42
2011-01-05,82.77,102.87,120.35,106.70,128.98,128.61,86.01,116.37,135.51,97.75
2011-01-06,83.09,102.38,119.32,106.95,130.06,128.83,85.40,116.25,135.96,97.37
2011-01-07,83.46,101.62,119.31,107.42,130.42,128.88,86.29,115.74,136.06,97.72


### 1.4 Calculate log returns

In [ ]:
# Calculate daily log returns
df_log_returns = np.log(df_EER / df_EER.shift(1))

df_log_returns.dropna(inplace=True)  # created in the shifting

df_log_returns.to_pickle("df_log_returns.pkl")

### 1.5 Calculate cumulative 5 day windows

In [54]:
# Future returns
# At index t, we want the sum of t+1, t+2, t+3, t+4, t+5 returns
df_future_returns = df_log_returns.rolling(window=5, min_periods=5).sum().shift(-5)
df_future_returns.dropna(inplace=True)

# Past returns
# At index t, we want the sum of t-1, t-2, t-3, t-4, t-5 returns
df_past_returns = df_log_returns.rolling(window=5, min_periods=5).sum().shift(1)
df_past_returns.dropna(inplace=True)


# Merge future and past returns DataFrames into df_log_returns, aligning on date index.
df_log_returns = df_log_returns.join(df_future_returns.add_suffix('_future'), how='inner')
df_log_returns = df_log_returns.join(df_past_returns.add_suffix('_past'), how='inner')

### 1.6 Get sentiment labels

Based of future returns:

For each timestep:
- Top 3 (30%) -> "appreciation"
- Middle 4 (40%) -> "unchanged"
- Bottom 3 (30%) -> "depreciation"

In [55]:
# Get list of currency codes (G10 currencies)
currency_codes = ['USD', 'EUR', 'JPY', 'GBP', 'CAD', 'AUD', 'CHF', 'SEK', 'NOK', 'NZD']

# Initialize label columns for each currency
for currency in currency_codes:
    df_log_returns[f'{currency}_label'] = None

# For each date (row), rank currencies by their future returns and assign labels
for date in df_log_returns.index:
    # Get future returns for this date
    future_returns = {}
    for currency in currency_codes:
        value = df_log_returns.loc[date, f'{currency}_future']
        if pd.notna(value):
            future_returns[currency] = value

    # Get past returns for this date
    past_returns = {}
    for currency in currency_codes:
        value = df_log_returns.loc[date, f'{currency}_past']
        if pd.notna(value):
            past_returns[currency] = value
    
    # Rank currencies by future returns (highest to lowest)
    sorted_currencies_future = sorted(future_returns.items(), key=lambda x: x[1], reverse=True)

    # Rank currencies by past returns (highest to lowest)
    sorted_currencies_past = sorted(past_returns.items(), key=lambda x: x[1], reverse=True)
    
    # Assign labels based on ranking
    # Top 3 (30%) -> "appreciation"
    # Middle 4 (40%) -> "unchanged"
    # Bottom 3 (30%) -> "depreciation"
    for i, (currency, _) in enumerate(sorted_currencies_future):
        if i < 3:  # Top 3 (0, 1, 2)
            df_log_returns.loc[date, f'{currency}_future_label'] = 'appreciation'
        elif i >= 7:  # Bottom 3 (7, 8, 9)
            df_log_returns.loc[date, f'{currency}_future_label'] = 'depreciation'
        else:  # Middle 4 (3, 4, 5, 6)
            df_log_returns.loc[date, f'{currency}_future_label'] = 'unchanged'
    
    for i, (currency, _) in enumerate(sorted_currencies_past):
        if i < 3:  # Top 3 (0, 1, 2)
            df_log_returns.loc[date, f'{currency}_past_label'] = 'appreciation'
        elif i >= 7:  # Bottom 3 (7, 8, 9)
            df_log_returns.loc[date, f'{currency}_past_label'] = 'depreciation'
        else:  # Middle 4 (3, 4, 5, 6)
            df_log_returns.loc[date, f'{currency}_past_label'] = 'unchanged'

# Only keep labels
df_labels = df_log_returns[
    [f'{currency}_future_label' for currency in currency_codes] + 
    [f'{currency}_past_label' for currency in currency_codes] 
]


In [56]:
# Get rid of time info
df_labels.index = df_labels.index.normalize()

# Have date as a column
df_labels = df_labels.reset_index()

df_labels.columns

Index(['date', 'USD_future_label', 'EUR_future_label', 'JPY_future_label',
       'GBP_future_label', 'CAD_future_label', 'AUD_future_label',
       'CHF_future_label', 'SEK_future_label', 'NOK_future_label',
       'NZD_future_label', 'USD_past_label', 'EUR_past_label',
       'JPY_past_label', 'GBP_past_label', 'CAD_past_label', 'AUD_past_label',
       'CHF_past_label', 'SEK_past_label', 'NOK_past_label', 'NZD_past_label'],
      dtype='object')

### 1.7 Assign labels to news articles
- df_labels - labels for sentiment for each valid day
- df_news_future and past - mapping between all real dates and valid dates based on the way we backfill
- df_news - now will contain these new sentiment labels joined using the dates from the news_future/past

In [57]:
# maps news date to future trading date
df_news_future = pd.merge_asof(
    df_news[['Date']],
    df_labels[['date']],
    left_on='Date',
    right_on='date',
    direction='forward'
).rename(columns={'Date': 'news_date', 'date': 'trading_date_future'})


# maps news date to past trading date
df_news_past = pd.merge_asof(
    df_news[['Date']],
    df_labels[['date']],
    left_on='Date',
    right_on='date',
    direction='backward'
).rename(columns={'Date': 'news_date', 'date': 'trading_date_past'})

# Perform a concat of df_news and df_news_future and df_news_past
# Example: Only concat 'news_date' from df_news_future, and 'trading_date_past' from df_news_past
df_news = pd.concat([
    df_news,
    df_news_future[['trading_date_future']],
    df_news_past[['trading_date_past']]
], axis=1)


# Merge the future labels into the news dataframe
future_cols = ['date'] + [col for col in df_labels.columns if col.endswith('_future_label')]
df_labels_future = df_labels[future_cols]

df_news = df_news.merge(df_labels_future, left_on='trading_date_future', right_on='date', how='left')
df_news = df_news.drop(columns=['date'])  # Drop the date column as we don't need it

# Merge the past labels into the news dataframe
past_cols = ['date'] + [col for col in df_labels.columns if col.endswith('_past_label')]
df_labels_past = df_labels[past_cols]

df_news = df_news.merge(df_labels_past, left_on='trading_date_past', right_on='date', how='left')
df_news = df_news.drop(columns=['date'])  # Drop the date column as we already have trading_date

### 1.8 Cleaning final DataFrame
- drop rows with nulls
- Removing unnecessary columns

In [58]:
df_news = df_news.rename(columns={'trading_date_future': 'Trading Date'})  # Rename trading_date_future to trading_date

# Only keep title, full text and all labels
df_news = df_news[['Title', 'Date', 'Full Text', 'mentioned_currencies', 'Trading Date', *future_cols[1:], *past_cols[1:]]]

df_news = df_news.dropna()

df_news.to_pickle("df_news.pkl")

df_news

,Title,Date,Full Text,mentioned_currencies,Trading Date,USD_future_label,EUR_future_label,JPY_future_label,GBP_future_label,CAD_future_label,...,USD_past_label,EUR_past_label,JPY_past_label,GBP_past_label,CAD_past_label,AUD_past_label,CHF_past_label,SEK_past_label,NOK_past_label,NZD_past_label
4,EUR/CHF Finds Support,2011-01-12,Sha...,"[EUR, CHF]",2011-01-12,depreciation,appreciation,appreciation,unchanged,depreciation,...,appreciation,depreciation,unchanged,appreciation,appreciation,depreciation,depreciation,unchanged,unchanged,unchanged
5,Risk Reward Ratios,2011-01-18,Sha...,"[EUR, USD]",2011-01-18,unchanged,appreciation,unchanged,depreciation,depreciation,...,depreciation,appreciation,unchanged,unchanged,unchanged,depreciation,depreciation,appreciation,unchanged,appreciation
6,How to Identify a Currency Pair that is in a T...,2011-01-19,Sha...,"[EUR, USD]",2011-01-19,unchanged,appreciation,unchanged,depreciation,unchanged,...,depreciation,appreciation,depreciation,appreciation,unchanged,unchanged,depreciation,unchanged,unchanged,appreciation
7,EURUSD Flashes a Bearish Pattern,2011-01-19,Sha...,"[EUR, USD]",2011-01-19,unchanged,appreciation,unchanged,depreciation,unchanged,...,depreciation,appreciation,depreciation,appreciation,unchanged,unchanged,depreciation,unchanged,unchanged,appreciation
8,Should you Trade the Trend or Trade the Double...,2011-01-24,Sha...,"[EUR, USD]",2011-01-24,unchanged,unchanged,unchanged,depreciation,depreciation,...,depreciation,appreciation,appreciation,unchanged,unchanged,unchanged,depreciation,appreciation,unchanged,depreciation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25970,Dow Sustains Losses but Nasdaq 100 Hits New Hi...,2024-05-23,Sha...,"[USD, AUD]",2024-05-23,unchanged,depreciation,unchanged,unchanged,depreciation,...,unchanged,unchanged,depreciation,appreciation,depreciation,unchanged,depreciation,unchanged,appreciation,appreciation
25971,"US Dollar Technical Outlook: EUR/USD, GBP/USD,...",2024-05-23,Sha...,"[EUR, USD, GBP, NZD]",2024-05-23,unchanged,depreciation,unchanged,unchanged,depreciation,...,unchanged,unchanged,depreciation,appreciation,depreciation,unchanged,depreciation,unchanged,appreciation,appreciation
25972,BoJ Faces Dilemma: Hiking Rates into Economic ...,2024-05-24,Sha...,"[USD, JPY]",2024-05-24,depreciation,depreciation,unchanged,depreciation,unchanged,...,unchanged,unchanged,depreciation,appreciation,depreciation,unchanged,depreciation,unchanged,appreciation,appreciation
25973,EUR/CHF IG Client Sentiment: Our data shows tr...,2024-05-24,Sha...,"[EUR, CHF]",2024-05-24,depreciation,depreciation,unchanged,depreciation,unchanged,...,unchanged,unchanged,depreciation,appreciation,depreciation,unchanged,depreciation,unchanged,appreciation,appreciation


### 1.9 Train Test Split
- 200 examples for final eval
- Otherwise 80/20 train test split

In [ ]:
df_news = pd.read_pickle("df_news.pkl")  # IMPORTANT - TODO CHANGE ME

df_news_before_2020 = df_news[df_news['Date'] < pd.to_datetime('2020-01-01', utc=True)]     # we train the model on this for now
# df_news_after_2020 = df_news[df_news['Date'] >= pd.to_datetime('2020-01-01', utc=True)]   # we use this for the trading strategy

df_rest, df_eval = train_test_split(df_news_before_2020, test_size=200, random_state=42)
df_train, df_test = train_test_split(df_rest, test_size=0.2, random_state=42)



print("Size of train set: ", len(df_train))
print("Size of test set: ", len(df_test))
print("Size of eval set: ", len(df_eval))

Size of train set:  2
Size of test set:  2
Size of eval set:  200


## 2 Generate Prompt

In [11]:
def generate_prompt(row):
    title = row.get('Title', '')
    text = row.get('Full Text', '')
    currencies = row.get('mentioned_currencies')

    target_currencies = ''
    for c in currencies:
        target_currencies += f'{c}_past: "appreciation, depreciation, or unchanged",\n'
        target_currencies += f'{c}_future: "appreciation, depreciation, or unchanged",\n'
    target_currencies = target_currencies.strip().rstrip(",") # Remove last comma

    # Same structure as per paper
    return (
        f"Title: {title}\n"
        f"Text: {text}\n\n"
        "Instructions:\n"
        "Objective: For each mentioned currency, answer the following questions:\n"
        "- What has been the current/past movement of the currency (appreciation, depreciation, or unchanged)?\n"
        "- What is the future expectation for the currency (appreciation, depreciation, or unchanged)?\n\n"
        "You must answer these two questions for each of the following currencies mentioned in the article:\n"
        f"{target_currencies}\n\n"
        "Output Format:\n"
        "- Important: Provide your answer in separate rows for each currency as shown above.\n"
        "- Do not combine multiple currencies in the same row.\n"
        '- Each currency should have its own line with "_past" or "_future" specified.\n\n'
        "Example:\n"
        '- If the article states, "The EUR is expected to appreciate," the output should be:\n'
        '    EUR_past: "unchanged",\n'
        '    EUR_future: "appreciation"\n'
        '- If the article states, "EUR/USD depreciated last week," the output should be:\n'
        '    EUR_past: "depreciation",\n'
        '    USD_past: "appreciation"\n'
        '- If only future movements are mentioned for a currency, the past movement should be labelled as "unchanged" and vice versa.\n\n'
        "Currency Pair Interpretation:\n"
        "- If currencies are discussed in pairs, interpret as follows:\n"
        '    - If "EUR/USD appreciated," label EUR_past as "appreciation" and USD_past as "depreciation".\n'
        '    - If "EUR/USD depreciated," label EUR_past as "depreciation" and USD_past as "appreciation".\n\n'
        "Synonyms:\n"
        "- Recognize the following synonyms for each currency:\n"
        "- **EUR**: EUR, Euro\n"
        "- **USD**: USD, Dollar, Dollars, US Dollar, US-Dollar, U.S. Dollar, US Dollars, US-Dollars, U.S. Dollars, Greenback\n"
        "- **JPY**: JPY, Yen, Japanese Yen\n"
        "- **GBP**: GBP, Pound, Pounds, Sterling, British Pound, British Pounds\n"
        "- **AUD**: AUD, Australian Dollar, Australian Dollars, Aussie\n"
        "- **CAD**: CAD, Canadian Dollar, Canadian Dollars\n"
        "- **CHF**: CHF, Swiss Franc, Swiss Francs, Swissie\n"
        "- **NZD**: NZD, New Zealand Dollar, New Zealand Dollars, Kiwi\n"
        "- **NOK**: NOK, Norwegian Krone, Norwegian Kroner\n"
        "- **SEK**: SEK, Swedish Krona, Swedish Kronor\n"
        "Answer below in the given format:"
    )

## 3 Model setup

In [ ]:
load_dotenv()
login(token=os.getenv("HF_TOKEN"))

# 'meta-llama/Llama-3.1-8B-Instruct' - real
# "meta-llama/Llama-3.2-1B-Instruct" - for local testing as smallest possible model
model_id =  "meta-llama/Llama-3.1-8B-Instruct"

# quntisation config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,      # "Double Quantization"
    bnb_4bit_quant_type="nf4",           # 4-bit NormalFloat data type
    bnb_4bit_compute_dtype=torch.bfloat16 # Compute in bfloat16 for stability
)

# load tokeniser
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length = 8192
tokenizer.pad_token = tokenizer.eos_token # Llama has no default pad token
tokenizer.padding_side = "right"  # TODO Check this

# load model 
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically puts model on GPU
    dtype=torch.bfloat16
)

# Move model to GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    model = model.to(device)
    print(f"Model explicitly loaded onto: {device}")
else:
    device = torch.device("cpu")
    model = model.to(device)
    print("CUDA not available. Model loaded onto CPU.")

model.config.use_cache = False
model.config.pretraining_tp = 1

# Prepare for training 
model = prepare_model_for_kbit_training(model)

# LoRA config 
# Params from Table A.1
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",           # TODO Check this
    task_type="CAUSAL_LM", # TODO Check this
    
    # inject low-rank adaptation matrices into all linear layers TODO check this
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj", # Attention projections
        "gate_proj", "up_proj", "down_proj"     # MLP projections
    ]
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Model explicitly loaded onto: cuda:0


## 4 LLM Fine Tuning
- Stopping criterion is used
    - Optimisises for least loss in the validation stage rather than most traning epochs
    - So if the model with best validation loss is in epoch 1 or 2, then the weights in epoch 3 will be discarded
    - Used to prevent overfitting due to this being a small dataset
    - Stops traning if the validation loss stagnates due to overfitting

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,      # TODO Check this
    per_device_eval_batch_size=1,       # TODO Check this
    gradient_accumulation_steps=32, 
    optim="paged_adamw_32bit",          # 
    save_steps=50,                      # TODO get better number
    learning_rate=1e-5,                 #  Note: significantly lower than standard
    weight_decay=0.1,                   #  High weight decay
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,                  # TODO apparenly this is the best for lora??? - not said in the paper
    warmup_ratio=0.0,                   # 
    lr_scheduler_type="cosine",         #                  
    save_strategy="steps",              # for early stopping   (could be epoch)
    eval_strategy="steps",              # for early stopping   (could be epoch)
    load_best_model_at_end=True,         # for early stopping
    metric_for_best_model="eval_loss",   # for early stopping
    greater_is_better=False,     # less loss is better
    logging_steps=10,                   # TODO get a better number
    group_by_length=True,
    report_to="none"                    # Disable wandb unless needed
)


df_train = Dataset.from_pandas(df_train)
df_test = Dataset.from_pandas(df_test)

trainer = SFTTrainer(
    model=model,
    train_dataset=df_train, # Ensure this is loaded
    eval_dataset=df_test,
    peft_config=peft_config,
    formatting_func=generate_prompt,
    processing_class=tokenizer,
    args=training_args,
    # packing=False,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)] # to stop after epoch 1 if validaiton loss gets worse
)


trainer.train()

trainer.model.save_pretrained('llama_3.1_8B_finetuned')
print("Model saved.")

Truncating eval dataset: 100%|██████████| 2/2 [00:00<00:00, 243.13 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.
/mnt/c/Users/Amogh/OneDrive - University of Cambridge/Programming-New/Part-II-Project/venv_wsl/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


Model saved.


## 5 Evaluate

### 5.1 Predict sentiment
- Gets the sentiment for a single article
- Used for evaulation

In [ ]:
def get_sentiment(row, model, tokenizer):
    prompt = generate_prompt(row)

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,  # to avoid crashing model due to very large article
        max_length=8192
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=512,     # only needs to generate enough for sentiment
            temperature=0.1,        # incase there was sampling
            do_sample=False,        # no sampling - so no randomness
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()    # skips over prompt

    # Parse response to get labels into a dict
    sentiment = {}
    for line in response.split('\n'):
        if line.strip():
            currency, label = line.split(':')
            currency = currency.strip()
            label = label.strip()
            sentiment[currency] = label

    return sentiment

### 5.2 Get evaulation statistics

In [ ]:
currency_codes = ['EUR', 'USD', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'NZD', 'NOK', 'SEK']

all_actual = []
all_predictions = []

for i, row in df_eval.iterrows():
    sentiment = get_sentiment(row, model, tokenizer)
    for c in currency_codes:
        for t in ['past', 'future']:
            all_actual.append(row[f'{c}_{t}_label'])
            all_predictions.append(sentiment.get(f'{c}_{t}', 'unchanged'))

    
    
accuracy = accuracy_score(all_actual, all_predictions)
f1 = f1_score(all_actual, all_predictions, average='macro')
precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(all_actual, all_predictions, labels=['appreciation', 'depreciation', 'unchanged'])

stats = {
    'accuracy': accuracy,
    'f1': f1,
    'precision_per_class': dict(zip(['appreciation', 'depreciation', 'unchanged'], precision_per_class)),
    'recall_per_class': dict(zip(['appreciation', 'depreciation', 'unchanged'], recall_per_class)),
    'f1_per_class': dict(zip(['appreciation', 'depreciation', 'unchanged'], f1_per_class)),
    'support_per_class': dict(zip(['appreciation', 'depreciation', 'unchanged'], support_per_class))
}

report = classification_report(all_actual, all_predictions)

print(stats)

print()
print()

print(report)

{'accuracy': 1.0, 'f1': 1.0, 'precision_per_class': {'appreciation': np.float64(1.0), 'depreciation': np.float64(1.0), 'unchanged': np.float64(1.0)}, 'recall_per_class': {'appreciation': np.float64(1.0), 'depreciation': np.float64(1.0), 'unchanged': np.float64(1.0)}, 'f1_per_class': {'appreciation': np.float64(1.0), 'depreciation': np.float64(1.0), 'unchanged': np.float64(1.0)}, 'support_per_class': {'appreciation': np.int64(60), 'depreciation': np.int64(60), 'unchanged': np.int64(80)}}


              precision    recall  f1-score   support

appreciation       1.00      1.00      1.00        60
depreciation       1.00      1.00      1.00        60
   unchanged       1.00      1.00      1.00        80

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



## 6 Downstream Application
Using model trained on pre 2020 data to backtest on 2020-2024 market

### 6.1 Load data

In [ ]:
df_news = pd.read_pickle("df_news.pkl")

# TODO check if its inclusive
df_news = df_news[
    (df_news['Date'] >= pd.to_datetime('2020-01-01')) &
    (df_news['Date'] < pd.to_datetime('2024-07-01'))
]

# Reset index
df_news = df_news.reset_index(drop=True)

# Drop the labels - we will predict these
df_news = df_news[['Title', 'Full Text', 'mentioned_currencies', 'Trading Date']]

df_news

,Title,Full Text,mentioned_currencies,Trading Date
0,USD/CAD Rate Trades Below 1.3000 Ahead of FOMC...,Sha...,"[USD, CAD]",2020-01-01
1,"Swiss Franc Forecast: CHF/JPY, NZD/CHF",Sha...,"[USD, CHF, NZD]",2020-01-02
2,Crude Oil Prices Up as US Strike Kills Iran Qu...,Sha...,"[USD, JPY, AUD, NZD]",2020-01-02
3,Gold Prices Hold Near 4-Month Highs Despite US...,Sha...,"[EUR, USD]",2020-01-02
4,US Dollar Prices May Reverse Higher as Japanes...,Sha...,"[EUR, USD, JPY, GBP]",2020-01-02
...,...,...,...,...
9497,Dow Sustains Losses but Nasdaq 100 Hits New Hi...,Sha...,"[USD, AUD]",2024-05-23
9498,"US Dollar Technical Outlook: EUR/USD, GBP/USD,...",Sha...,"[EUR, USD, GBP, NZD]",2024-05-23
9499,BoJ Faces Dilemma: Hiking Rates into Economic ...,Sha...,"[USD, JPY]",2024-05-24
9500,EUR/CHF IG Client Sentiment: Our data shows tr...,Sha...,"[EUR, CHF]",2024-05-24


### 6.2 Load LLM

In [ ]:
# Path to the saved directory
model_dir = "llama_3.1_8B_finetuned"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)

print("tokeniser and model loaded")

### 6.3 Make inferences for all articles

In [ ]:
print("Getting sentiment predictions for all articles...")

sentiment_predictions = []

for idx, row in tqdm(df_news.iterrows(), total=len(df_news), desc="Processing articles"):
    sentiment = get_sentiment(row, model, tokenizer)
    sentiment_predictions.append(sentiment)
    df_news.at[idx, 'sentiment_predictions'] = sentiment

### 6.4 Daily Sentiment Score Generation


$$S_{i, t} = round(log(1+CountAppreciation_{i, t}) - log(1+CountDepreciation_{i, t}))$$

where $CountAppreciation_{i, t}$ is the number of articles published on day $t$ for which the model assigns the **future label** of currency $i$ to “appreciation”

$round$ function is defined below:
$$
\widehat{S}_{i,t} = 
\begin{cases} 
+1, & \text{if } S_{i,t} > 0, \\
0, & \text{if } S_{i,t} = 0, \\
-1, & \text{if } S_{i,t} < 0.
\end{cases}
$$

In [ ]:
# Initialise dict (currency, date)
data_for_S = {}

# Get list of currency codes
currency_codes = ['USD', 'EUR', 'JPY', 'GBP', 'CAD', 'AUD', 'CHF', 'SEK', 'NOK', 'NZD']

# Group by date
for date, group in df_news.groupby('Trading Date'):
    # For each currency
    for currency in currency_codes:
        # Count appreciation and depreciation for this currency on this date
        count_appreciation = 0
        count_depreciation = 0
        
        for _, row in group.iterrows():
            # Get the future label prediction for this currency
            future_key = f'{currency}_future'
            prediction = row['sentiment_predictions'].get(future_key, 'unchanged')
            
            if prediction == 'appreciation':
                count_appreciation += 1
            elif prediction == 'depreciation':
                count_depreciation += 1
        
        # Calculate S_{i, t} = log(1 + CountAppreciation) - log(1 + CountDepreciation)
        S_value = np.log(1 + count_appreciation) - np.log(1 + count_depreciation)

        # Round to 1 if positive, 0 if 0, -1 if negative
        if S_value > 0:
            S_value = 1
        elif S_value < 0:
            S_value = -1
        else:
            S_value = 0
        
        # Add to dict
        data_for_S[(currency, date)] = S_value


# Get list of dates
dates = sorted(set([date for _, date in data_for_S.keys()]))

# convert to a numpy matrix shape (currencies, dates)
S = np.array([[data_for_S[(currency, date)] for date in dates] for currency in currency_codes])

unique_dates_array = np.sort(pd.to_datetime(df_news['Trading Date'].unique()))
unique_dates_array = pd.to_datetime(unique_dates_array)
unique_dates_array = unique_dates_array.tz_localize(None)  # removes timezone info if present
unique_dates_array = unique_dates_array.normalize()        # sets time to midnight (removes time info)

print(S)
print()
print()
print(unique_dates_array)


[[ 0 -1  1 ... -1  1  1]
 [ 1  0  0 ... -1  0  1]
 [-1  0  1 ...  1  0  0]
 ...
 [ 0 -1  1 ... -1 -1  0]
 [ 1 -1  1 ... -1 -1 -1]
 [ 1  1 -1 ... -1 -1  1]]
DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06',
               '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
               '2020-01-13', '2020-01-14',
               ...
               '2024-05-13', '2024-05-14', '2024-05-15', '2024-05-16',
               '2024-05-17', '2024-05-20', '2024-05-21', '2024-05-22',
               '2024-05-23', '2024-05-24'],
              dtype='datetime64[ns]', length=1145, freq=None)


### 6.5 Get weights

$$
w_{i,t:t+1} = 
\begin{cases} 
\frac{\widehat{S}_{i,t}}{\sum_{j:\widehat{S}_{j,t}>0} \widehat{S}_{j,t}}, & \text{if } \widehat{S}_{i,t} > 0 \text{ and } \sum_{j:\widehat{S}_{j,t}>0} \widehat{S}_{j,t} > 0 \\
-\frac{\widehat{S}_{i,t}}{\sum_{j:\widehat{S}_{j,t}<0} |\widehat{S}_{j,t}|}, & \text{if } \widehat{S}_{i,t} < 0 \text{ and } \sum_{j:\widehat{S}_{j,t}<0} |\widehat{S}_{j,t}| > 0 \\
0, & \text{otherwise.}
\end{cases}
$$

Long on positive sentiment scores

Short negative sentiment scores

Value of each position is proprtional to sentiment score 

Value of all short positions are the same as all the long positions for any given day

Held for 1 day (close of day t until close of day t+1)

if no new article was published on the next day, the previous sentiment signal is retained

In [77]:
weights = np.zeros_like(S, dtype=float)

positive_mask = S > 0
negative_mask = S < 0

sum_positive = np.sum(S * positive_mask, axis=0, keepdims=True)
sum_negative_abs = np.sum(np.abs(S) * negative_mask, axis=0, keepdims=True)

valid_positive_sums = sum_positive > 0
valid_negative_sums = sum_negative_abs > 0

with np.errstate(divide='ignore', invalid='ignore'):
    positive_weights = np.where(valid_positive_sums, S / sum_positive, 0)
    negative_weights = np.where(valid_negative_sums, S / sum_negative_abs, 0)

weights = np.where(positive_mask & valid_positive_sums, positive_weights, weights)
weights = np.where(negative_mask & valid_negative_sums, negative_weights, weights)

weights

array([[ 0.        , -0.25      ,  0.16666667, ..., -0.16666667,
         0.33333333,  0.25      ],
       [ 0.25      ,  0.        ,  0.        , ..., -0.16666667,
         0.        ,  0.25      ],
       [-1.        ,  0.        ,  0.16666667, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        , -0.25      ,  0.16666667, ..., -0.16666667,
        -0.25      ,  0.        ],
       [ 0.25      , -0.25      ,  0.16666667, ..., -0.16666667,
        -0.25      , -0.25      ],
       [ 0.25      ,  0.33333333, -0.5       , ..., -0.16666667,
        -0.25      ,  0.25      ]], shape=(10, 1145))

### 6.6 Get the market data

In [78]:
df_log_returns = pd.read_pickle("df_log_returns.pkl")

df_log_returns = df_log_returns[df_log_returns.index.isin(unique_dates_array)]

print(df_log_returns.shape)
print(len(unique_dates_array))

(1145, 10)
1145


### 6.7 Execute strategy

In [ ]:
df_log_returns_T = df_log_returns.transpose()
log_returns = df_log_returns_T.values   # turn it into a matrix

# Remove the last one as doesnt have future returns
W_active = weights[:, :-1]

# Allign for future returns
R_future = log_returns[:, 1:]

# Element wise multiplication
# Sum vertically for each day
daily_pnl = np.sum(W_active * R_future, axis=0)

[-0.00089039 -0.00234365 -0.00125497 ... -0.00268306 -0.00329367
  0.00029751]


date,2020-01-01,2020-01-02,2020-01-03,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,2020-01-13,2020-01-14,...,2024-05-13,2024-05-14,2024-05-15,2024-05-16,2024-05-17,2024-05-20,2024-05-21,2024-05-22,2024-05-23,2024-05-24
USD,0.0,0.002602,0.001598,-0.001798,0.001598,0.002890,0.001492,0.000993,-0.001291,0.000795,...,-0.000818,0.000455,-0.004100,-0.002835,0.002561,-0.000731,0.000457,0.002737,-0.001733,0.002370
EUR,0.0,-0.001632,-0.002043,0.003165,-0.000816,-0.004090,0.001331,-0.001638,0.003887,-0.001430,...,0.000992,0.000594,-0.000099,0.000594,0.000198,0.000693,0.000297,-0.001385,0.000594,0.000297
JPY,0.0,0.000407,0.008302,-0.000807,-0.002425,-0.001113,-0.006502,-0.001836,-0.003478,-0.001436,...,-0.001256,-0.003918,0.003219,0.001117,-0.003776,-0.000420,-0.003089,0.000422,-0.001547,-0.002255
GBP,0.0,0.002307,-0.004715,0.000289,0.000000,0.002119,-0.004726,0.005207,-0.010248,0.001262,...,0.000866,0.000769,0.001824,0.000288,0.002011,0.002103,0.001527,0.002666,0.000285,-0.000761
CAD,0.0,0.000484,0.001933,-0.000097,-0.001160,-0.000871,-0.002616,-0.000971,0.001747,-0.001844,...,0.000596,0.000496,0.001487,-0.000892,0.000496,0.000396,0.000495,-0.003371,0.000199,-0.003083
AUD,0.0,-0.001670,-0.004038,-0.002569,-0.008945,0.000399,0.000698,0.002689,0.003376,-0.000099,...,0.001907,-0.000858,0.001048,0.001237,0.000951,0.002467,-0.000379,-0.002088,-0.002188,-0.003339
CHF,0.0,-0.001842,0.001126,0.000511,-0.000409,0.003777,-0.001121,-0.002042,0.002552,0.003765,...,-0.000091,-0.001548,0.000273,-0.001823,-0.003565,-0.002200,-0.000184,-0.002942,0.000184,-0.001658
SEK,0.0,-0.003127,-0.001921,-0.003244,-0.001321,0.001829,-0.001727,-0.002239,0.000509,0.003051,...,-0.000886,-0.002328,0.004430,0.005181,-0.003304,0.003743,0.002743,-0.004944,0.003408,-0.000549
NOK,0.0,0.002133,0.000648,0.000185,-0.000926,-0.001391,-0.001114,-0.001208,-0.000558,-0.000652,...,-0.000106,0.000426,0.001913,-0.000212,0.001486,0.001695,0.002115,0.000000,0.005583,0.002099
NZD,0.0,-0.004743,-0.001748,-0.000778,-0.001363,0.002723,-0.002821,-0.000097,0.001849,-0.002531,...,0.000495,0.000297,0.003655,0.003740,0.002649,-0.000882,-0.001177,0.004017,0.002442,-0.001367


### 6.8 Evaluate results

Assume a zero cost portiolio so risk free rate is 0 

$$Annualized\ Return = mean\ daily\ return \times 252$$

$$Annualized\ Volatility = Standard\ Deviation\ of\ Daily\ Returns \times sqrt(252)$$

$$Sharpe\ Ratio = \frac{Annualized\ Return}{Annualized\ Volatility}$$

$$Maximum\ Drawdown =\ The\ largest\ peak\ to\ trough\ decline\ in\ cumulative\ returns$$

$$Transaction\ Costs = sum\ of\ absolute\ weight\ changes\ per\ day,\ averaged,\ then\ annualized

In [92]:
ANNUALIZATION_FACTOR = 252    # Number of trading days per year


# Annualized Return
ann_return = np.mean(daily_pnl) * ANNUALIZATION_FACTOR
ann_return_pc = ann_return * 100  # percent

# Annualized Volatility
ann_vol = np.std(daily_pnl, ddof=1) * np.sqrt(ANNUALIZATION_FACTOR)
ann_vol_pc = ann_vol * 100        # Convert to percentage

# Sharpe Ratio
if ann_vol != 0:
    sharpe_ratio = ann_return / ann_vol
else:
    sharpe_ratio = 0.0

# Maximum Drawdown
cumulative_returns = np.cumsum(daily_pnl)
running_max = np.maximum.accumulate(cumulative_returns)
drawdown = running_max - cumulative_returns
max_drawdown = np.max(drawdown)
max_drawdown_pc = max_drawdown * 100  # Convert to percentage

# Transaction Costs (Rebalancing Frequency)
weight_changes = np.abs(np.diff(weights, axis=1))  # Change from day t to t+1
daily_turnover = np.sum(weight_changes, axis=0)    # Total turnover per day
rebalancing_frequency = np.mean(daily_turnover) * ANNUALIZATION_FACTOR

print(f"Annualized Return:    {ann_return_pc:.2f}%")
print(f"Annualized Volatility:{ann_vol_pc:.2f}%")
print(f"Sharpe Ratio:         {sharpe_ratio:.2f}")
print(f"Max Drawdown:         {max_drawdown_pc:.2f}%")
print(f"Rebalancing Freq:     {rebalancing_frequency:.2f}")

Annualized Return:    -1.04%
Annualized Volatility:5.07%
Sharpe Ratio:         -0.21
Max Drawdown:         9.39%
Rebalancing Freq:     678.25
